In [2]:
# imports
import requests
import json
import os
import sys
import platform
import time
import math
import datetime
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

In [3]:
# generic store data to file function
def store_data(data, file, mode='w', toJson=False):
    if toJson:
        data = json.dumps(data)
    with open(file, mode, encoding='utf-8') as fp:
        result = fp.write(data)
        return result
    
# generic load data from file function
def load_data(file, fromJson=False):
    if os.path.isfile(file):
        with open(file, 'r', encoding='utf-8', errors="ignore") as fp:
            data = fp.read()
            if fromJson:
                data = json.loads(data)
            return data
    else:
        return 'file not found'

# test text
#print(store_data('Hello', '../data/repositories/mlart/test.txt'))
#print(load_data('../data/repositories/mlart/test.txt'))

# test json
#print(store_data({'msg':'Hello World'}, '../data/repositories/mlart/test.json', toJson=True))
#print(load_data('../data/repositories/mlart/test.json', fromJson=True))

In [4]:
# helper function to create folder create_folder
def create_folder(path):
    if not os.path.exists(os.path.dirname(path)):
        try:
            os.makedirs(os.path.dirname(path))
            print(path + ' created')
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

In [5]:
# scan text for predefined terms

text = 'We use LSTM for anomaly and object detection. As Convolutional Neural Networks are great for ML.'

pd_ml_terms = pd.read_csv('../data/patterns/ml_terms.csv')
ml_terms = pd_ml_terms['Term'].tolist()
ml_slugs = pd_ml_terms['Slug'].tolist()
ml_slugs = [x for x in ml_slugs if str(x) != 'nan']
ml_tags = pd_ml_terms['Tag'].tolist()
ml_tags = [x for x in ml_tags if str(x) != 'nan']

#print(ml_tags)

ml_libs = pd.read_csv('../data/patterns/ml_libraries.csv')
ml_libs = ml_libs['Python Package'].tolist()

def match_text(haystack, needles, toLower = False, unique = True):
    
    if toLower == True:
        haystack = haystack.lower()
        needles = [x.lower() for x in needles]
    
    if unique == True:
        matches = {x for x in needles if x in haystack}
        matches = list(matches)
    else:
        matches = [x for x in needles if x in haystack]
    
    return matches

def match_tags(haystack):
    df = pd.read_csv('../data/patterns/ml_terms.csv')
    tags = []
    
    df.set_index('Term', inplace = True)
    for item in haystack:
        try:
            tag = df.loc[item].get('Tag')
            if not 'nan' in str(tag):
                tags.append(tag)
        except:
            pass
        
    df.set_index('Slug', inplace = True)
    for item in haystack:
        try:
            tag = df.loc[item].get('Tag')
            if not 'nan' in str(tag):
                tags.append(str(tag))
        except:
            pass
        
    #if 'ANN' in tags or 'CNN' in tags or 'RNN' in tags:
    #    tags.remove('NN')
    
    return list(tags)

#ml_slugs, ml_terms, ml_libs, match_text(haystack, needles, toLower = False, unique = True)
needles = {
    'ml_slugs': ml_slugs,
    'ml_terms': ml_terms,
    'ml_libs': ml_libs,
}
needles_need_str_lower = {
    'ml_slugs': False,
    'ml_terms': True,
    'ml_libs': False,
}

matches = []

matches.extend(match_text(text, ml_terms, True))
matches.extend(match_text(text, ml_slugs, False))
print('matches', matches)

tags = match_tags(matches)
print('tags', tags)

matches ['detect', 'object detection', 'anomaly', 'lstm', 'convolutional neural network', 'neural network', 'ML']
tags ['Object Detection', 'LSTM', 'CNN', 'NN', 'ML']


In [6]:
# get file modifictaion date
# https://stackoverflow.com/questions/237079/how-to-get-file-creation-modification-date-times-in-python

def creation_date(path_to_file, datetime = True):
    """
    Try to get the date that a file was created, falling back to when it was
    last modified if that isn't possible.
    See http://stackoverflow.com/a/39501288/1709587 for explanation.
    """
    if platform.system() == 'Windows':
        timestamp = os.path.getctime(path_to_file)
    else:
        stat = os.stat(path_to_file)
        try:
            timestamp = stat.st_birthtime
        except AttributeError:
            # We're probably on Linux. No easy way to get creation dates here,
            # so we'll settle for when its content was last modified.
            timestamp = stat.st_mtime
        
    if datetime == True:
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(timestamp))

    return timestamp
    
folder_base = '../data/repositories/kaggle/competitions/c/'
folder = '3d-object-detection-for-autonomous-vehicles/notebooks/asimandia/lyft3d-inference-kernel/'
notebook = 'notebook_02.html'
kernel = 'kernel.html'
print(creation_date(folder_base+folder+notebook))

2020-12-12 16:08:03


In [7]:
# clear text formatting
def clear_text(text):
    return text.replace('\n',' ').replace('\r','').replace('¶','').strip()

In [7]:
# scrape competition

folder_base = '../data/repositories/kaggle/competitions/c/'
folder = '3d-object-detection-for-autonomous-vehicles/'
dataset = 'dataset.html'

def scrape_competition(html):
    soup = BeautifulSoup(html, 'html.parser')
    meta = {}
    
    meta['title'] = soup.find('h1', class_="competition-header__title").text.strip()
    meta['subtitle'] = soup.find('h2', class_="competition-header__subtitle").text.strip()
    meta['type'] = soup.find('p', class_="competition-header__classification-text").text.strip()
    meta['organisation'] = soup.find('span', class_="competition-header__organization-name")
    if meta['organisation'] == None:
        meta['organisation'] = ''
    else:
        meta['organisation'] = meta['organisation'].text.strip()
    temp = soup.find_all('li', class_="horizontal-list-item horizontal-list-item--bullet horizontal-list-item--default")
    for item in temp:
        #print(item.text)
        if 'team' in item.text:
            meta['teams'] = item.text.replace('teams','').replace('team','').strip()
        if 'ago' in item.text:
            #item = BeautifulSoup(item, 'html.parser')
            item = item.select('li>span>span')[0]
            meta['date_closed'] = item.get('title')
    
    meta['description'] = clear_text(soup.find('div', class_="competition-overview__content").text)
    meta['tags'] = [x.text.strip() for x in soup.find_all('span', class_="CategoryButton_CategoryName-sc-c10946 jFsDhg")]
    
    
    

    return meta

html = load_data(folder_base+folder+dataset)
if 'file not found' in html:
    print(html)
meta = scrape_competition(html)
print(meta)
#store_data(links, folder_base+folder+out, toJson=True)

{'title': 'Lyft 3D Object Detection for Autonomous Vehicles', 'subtitle': 'Can you advance the state of the art in 3D object detection?', 'type': 'Featured prediction Competition', 'organisation': 'Lyft', 'teams': '547', 'date_closed': 'Wed Nov 13 2019 00:59:00 GMT+0100 (Mitteleuropäische Normalzeit)', 'description': 'Self-driving technology presents a rare opportunity to improve the quality of life in many of our communities. Avoidable collisions, single-occupant commuters, and vehicle emissions are choking cities, while infrastructure strains under rapid urban growth. Autonomous vehicles are expected to redefine transportation and unlock a myriad of societal, environmental, and economic benefits. You can apply your data analysis skills in this competition to advance the state of self-driving technology. Lyft, whose mission is to improve people’s lives with the world’s best transportation, is investing in the future of self-driving vehicles. Level 5, their self-driving division, is wo

In [ ]:
# iterate competions

url = 'https://www.kaggle.com/c/'
folder_base = '../data/repositories/kaggle/competitions/c/'
folder = '3d-object-detection-for-autonomous-vehicles/'
dataset = 'dataset.html'
file_out = 'meta.json'

quit = 0 # quit after n files processed / 0 ... no limit

folders = os.listdir(folder_base)
print('folder:', len(folders))
i = 0

runtime_start = time.time()

for folder in folders:
    subfolders = os.listdir(os.path.join(folder_base,folder))
    print('subfolder:', i, len(folders))
    #print(folder)
    #print('###', i, os.path.join(folder_base,folder,subfolder))
    path = os.path.join(folder_base,folder,dataset)
    i += 1

    if os.path.exists(path):
        print(path)
        html = load_data(path)
        if 'file not found' in html:
            print(html)
        meta = scrape_competition(html)
        meta['link'] = url+folder
        #print(meta)
        store_data(meta, os.path.join(folder_base,folder,file_out), toJson=True)
            
    if quit!=0 and i>quit:
        break

runtime_end = time.time()
print('runtime:', round(runtime_end - runtime_start, 3), 'seconds for', i, 'items')

In [12]:
# scrape datasets

folder_base = '../data/repositories/kaggle/datasets/'
folder = '4quant/depth-generation-lightfield-imaging/'
dataset = 'dataset.html'

def scrape_datasets(html):
    soup = BeautifulSoup(html, 'html.parser')
    meta = {}
    
    try:
        meta['title'] = soup.find('h1', class_="dataset-header-v2__title").text.strip()
        meta['subtitle'] = soup.find('h2', class_="dataset-header-v2__subtitle").text.strip()
        meta['date'] = soup.find('time').get('datetime')
        meta['type'] = soup.find('p', class_="dataset-header-v2__classification-text").text.strip()
        meta['organisation'] = soup.find('div', class_="dataset-header-v2__collaborators-wrapper")
        if meta['organisation'] == None:
            meta['organisation'] = ''
        else:
            meta['organisation'] = meta['organisation'].text.strip()
        temp = soup.find_all('li', class_="horizontal-list-item horizontal-list-item--bullet horizontal-list-item--default")
        for item in temp:
            #print(item.text)
            if 'team' in item.text:
                meta['teams'] = item.text.replace('teams','').replace('team','').strip()
            if 'ago' in item.text:
                #item = BeautifulSoup(item, 'html.parser')
                item = item.select('li>span>span')[0]
                meta['date_closed'] = item.get('title')

        meta['description'] = clear_text(soup.find('div', class_="Description_ContentWrapper-sc-xv68c4").text)
        meta['usability'] = soup.find('p', {'data-test': 'rating'}).text.strip()
        meta['license'] = soup.find('a', class_="QuickInfo_A-sc-wkm1up").text.strip()
        meta['tags'] = [x.text.strip() for x in soup.find_all('a', {'class': 'SimpleTag_Tag-sc-1m0qa3y'})]

        footer = soup.find('div', {'class', 'Footer_Wrapper-sc-1fi89lh'})
        items = footer.find_all('li')
        for item in items:
            if 'view' in item.text:
                meta['views'] = int(item.text.replace('view','').replace('s','').replace(',','').strip())
            if 'download' in item.text:
                meta['downloads'] = int(item.text.replace('download','').replace('s','').replace(',','').strip())
            if 'notebook' in item.text:
                meta['notebooks'] = int(item.text.replace('notebook','').replace('s','').replace(',','').strip())
            if 'topic' in item.text:
                meta['topics'] = int(item.text.replace('topic','').replace('s','').replace(',','').strip())
    except:
        pass
    
    return meta

html = load_data(folder_base+folder+dataset)
if 'file not found' in html:
    print(html)
meta = scrape_datasets(html)
print(meta)
#store_data(links, folder_base+folder+out, toJson=True)

{'title': 'Computational Imaging', 'subtitle': 'Lightfield (Lytro) and Stereo (Project Tango) Datasets', 'date': '2018-07-10T16:58:22.407Z', 'type': 'Dataset', 'organisation': '4Quant', 'description': 'Context The data is based on images I have taken with my Lytro Illum camera (https://pictures.lytro.com/ksmader) they have been exported as image data and depth maps. The idea is to make and build tools for looking at Lytro Image data and improving the results Content The data are from the Lytro Illum and captured as 40MP images which are then converted to 5MP RGB+D images. All of the required data for several test images is provided The second datasets come from the Lenovo Phab2 (Project Tango) which utilizes dual image sensors to recreate point clouds of large 3D structures. These are provided as .ply and .obj datasets Acknowledgements The data is based on images I have taken with my Lytro Illum camera (https://pictures.lytro.com/ksmader). Inspiration  Build a neural network which auto

In [ ]:
# iterate datasets

url = 'https://www.kaggle.com/'
folder_base = '../data/repositories/kaggle/datasets/'
folder = '3d-object-detection-for-autonomous-vehicles/'
dataset = 'dataset.html'
file_out = 'meta.json'

quit = 0 # quit after n files processed / 0 ... no limit

folders = os.listdir(folder_base)
print('folder:', len(folders))
i = 0

runtime_start = time.time()

for folder in folders:
    projects = os.listdir(os.path.join(folder_base, folder))
    
    for project in projects:
        #subfolders = os.listdir(os.path.join(folder_base,folder,project))
        print('subfolder:', i, len(folders))
        #print(folder)
        #print('###', i, os.path.join(folder_base,folder,subfolder))
        path = os.path.join(folder_base,folder,project,dataset)
        i += 1

        if os.path.exists(path):
            print(path)
            html = load_data(path)
            if 'file not found' in html:
                print(html)
            meta = scrape_datasets(html)
            meta['link'] = url+folder+'/'+project
            #print(meta)
            #path = os.path.join(folder_base,folder,project,file_out)
            #print(path)
            store_data(meta, os.path.join(folder_base,folder,project,file_out), toJson=True)

        if quit!=0 and i>quit:
            break
    if quit!=0 and i>quit:
        break

runtime_end = time.time()
print('runtime:', round(runtime_end - runtime_start, 3), 'seconds for', i-1, 'items')

In [ ]:
# collect all meta.json into a single csv

folder_base = '../data/repositories/kaggle/competitions/'
file_meta = 'meta.json'
fp_csv = '../data/database/kaggle_competitions.csv'

# datasets
folder_base = '../data/repositories/kaggle/datasets/'
file_meta = 'meta.json'
fp_csv = '../data/database/kaggle_datasets.csv'

quit = 0 # quit after n files processed / 0 ... no limit

folders = os.listdir(folder_base)
print('folder:', len(folders))
i = 0

runtime_start = time.time()
df = pd.DataFrame()

runtime_start = time.time()
print('folder:', len(folders))
for folder in folders:
    projects = os.listdir(os.path.join(folder_base,folder))
    for project in projects:
        #subfolders = os.listdir(os.path.join(folder_base,folder))
        print(i)
        #print(folder)
        #print('###', i, os.path.join(folder_base,folder,subfolder))
        path = os.path.join(folder_base,folder,project,file_meta)
        i += 1

        if os.path.exists(path):
            print(path)
            data = load_data(path, fromJson=True)
            if 'file not found' in data:
                print(html)

            try:
                data['description'] = clear_text(data['description'])
            except:
                data['description'] = ''

            if 'date_closed' in data:
                # date (ignoring GMT+x)
                date_time_str = data['date_closed'].split('GMT')
                date_time_str = date_time_str[0].strip()
                date_time_obj = datetime.datetime.strptime(date_time_str, '%a %b %d %Y %H:%M:%S')
                data['date_closed'] = date_time_obj
            
            if 'date' in data:
                # date (ignoring GMT+x)
                date_time_str = data['date'].split('.')
                date_time_str = date_time_str[0].strip().replace('T',' ').replace('Z','')
                date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')
                data['date'] = date_time_obj

            df = df.append(data, ignore_index=True)

        if quit!=0 and i>quit:
            break
    if quit!=0 and i>quit:
        break
        
runtime_end = time.time()
print('runtime:', round(runtime_end - runtime_start, 3), 'seconds for', i, 'items')
print(df.shape)
print(df.head())
        
# drop columns
#df.drop(columns=['author', 'submission'], inplace=True)

df.to_csv(fp_csv, sep=';', index=False)

In [40]:
# scrape notebook content

folder_base = '../data/repositories/kaggle/competitions/c/'
folder1 = '3d-object-detection-for-autonomous-vehicles/notebooks/asimandia/lyft3d-inference-kernel/'
folder2 = 'aerial-cactus-identification/notebooks/abhinand05/in-depth-guide-to-convolutional-neural-networks/'
notebook = 'notebook_02.html'

def scrape_notebook_content(html):
    soup = BeautifulSoup(html, 'html.parser')
    meta = {}
    
    error = soup.find('h1', class_='not-found__header')
    if not error == None:
        meta['error'] = 'not found'
        return meta
    
    try:
        meta['author'] = meta['author'].get('href').replace('/','')
    except:
        meta['author'] = ''
        
    try:
        meta['title'] = soup.find('a', class_="KernelViewerContext_KernelTitle-sc-rdaqnd").text
    except:
        meta['title'] = ''
    
    try:
        meta['type'] = soup.find('span', class_="KernelViewerContext_KernelTypeInfo-sc-1l6fza6 kqxzvL").text.replace('using data from','').strip()
    except:
        meta['type'] = ''
    
    try:
        meta['sources'] = soup.find('a', class_="KernelViewerContext_DataSourceUrl-sc-1dm3ij9 lpoMHV").text.strip()
    except:
        meta['sources'] = ''
    
    try:
        meta['tags'] = soup.find('span', class_="KernelViewerContext_CategoriesWrapper-sc-8yrjj NgcTE").text.split('·')
        meta['tags'] = list(filter(None, meta['tags']))
        meta['tags'] = [x.replace('+1 more','') for x in meta['tags']]
        meta['tags'] = [x.replace('+2 more','') for x in meta['tags']]
        meta['tags'] = [x.replace('+3 more','') for x in meta['tags']]
        meta['tags'] = [x.replace('+4 more','') for x in meta['tags']]
    except:
        meta['tags'] = ''
    
    try:
        meta['votes'] = soup.find('span', class_="vote-button__vote-count").text
    except:
        meta['votes'] = ''
        
    try:
        meta['submission'] = soup.select('div.kernel-code-pane__submission-info-content')
        if len(meta['submission']) > 0:
            meta['submission'] = meta['submission'][0].text
    except:
        meta['submission'] = ''
        
    #meta['votes'] = soup.find('span', class_="vote-button__vote-count").text
    try:
        meta['license'] = soup.select('div.kernel-code-pane__subtitle>a')
        if len(meta['license']) > 0:
            meta['license'] = meta['license'][0].text
    except:
        meta['license'] = ''
    
    try:
        score = soup.select('div.kernel-code-pane__submission-score-value')
        if len(score) > 0:
            meta['score_private'] = score[0].text
        if len(score) > 1:
            meta['score_public'] = score[1].text
    except:
        pass
    
    try:
        temp = soup.find('span', class_="KernelViewerContext_KernelSubtitle-sc-rltxca esPWpV").select('span') #.text.split('·')
        #print (temp)
        for item in temp:
            #print(item)
            if 'views' in item.text:
                meta['views'] = int(''.join(filter(str.isdigit, item.text)))
            if 'GMT' in item.get('title', 'nan'):
                meta['date'] = item.get('title')
    except:
        pass
    
    return meta

html = load_data(folder_base+folder2+notebook)
if 'file not found' in html:
    print(html)
meta = scrape_notebook_content(html)
print(meta)
#store_data(links, folder_base+folder+out, toJson=True)

{'author': '', 'title': 'In-Depth Guide to Convolutional Neural Networks', 'type': 'Python notebook', 'sources': 'Aerial Cactus Identification', 'tags': ['beginner, exploratory data analysis, deep learning, cnn'], 'votes': '62', 'submission': 'Best Submission SuccessfulSubmitted by Abhinand a year ago', 'license': 'Apache 2.0', 'score_private': '0.9981', 'score_public': '0.9981', 'views': 4474, 'date': 'Sun Jun 16 2019 19:46:53 GMT+0200 (Mitteleuropäische Sommerzeit)'}


In [42]:
# scrape kernel content

folder_base = '../data/repositories/kaggle/competitions/c/'
folder1 = '3d-object-detection-for-autonomous-vehicles/notebooks/asimandia/lyft3d-inference-kernel/'
folder2 = 'airbnb-recruiting-new-user-bookings/notebooks/datadave/ndcg-score-r/'
notebook = 'notebook_02.html'
kernel = 'kernel.html'

def scrape_kernel_content(html, needles, snippet = False):
    soup = BeautifulSoup(html, 'html.parser')
    
    meta = {}
    
    try:
        if snippet == True:
            code = soup.find('div', {"id": "code"}).text
        else:
            code = soup.find('body').text
        #print(code)

        for key, value in needles.items():
            meta[key] = match_text(code, value, needles_need_str_lower[key])

        meta['ml_tags'] = match_tags(meta['ml_terms'] + meta['ml_slugs'])

        meta['description'] = soup.select('div.text_cell_render')
        #print(meta['description'])
        if len(meta['description']) > 0:
            meta['description'] = clear_text(meta['description'][0].text)
        else:
            meta['description'] = ''
    except:
        pass
    
    return meta

# test for non-kernel-file ('code' embedded in notebook.html)
html = load_data(folder_base+folder2+notebook)
if 'file not found' in html:
    print(html)
meta = scrape_kernel_content(html, needles)
print(meta)

# test for kernel-file ('code' in kernel.html)
html = load_data(folder_base+folder1+kernel)
if 'file not found' in html:
    print(html)
meta = scrape_kernel_content(html, needles)
print(meta)


#store_data(links, folder_base+folder+out, toJson=True)

{'ml_slugs': [], 'ml_terms': ['rank', 'deep learning', 'detect', 'classif', 'fit', 'filter', 'predict', 'train', 'training data'], 'ml_libs': [], 'ml_tags': ['DL', 'Classification'], 'description': ''}
{'ml_slugs': ['AI'], 'ml_terms': ['epoch', 'model', 'label', 'u-net', 'relu', 'layer', 'detect', 'loss', 'activation function', 'neural network', 'test data', 'fit', 'filter', 'predict', 'recommend', 'train', 'convolutional neural network', 'image segmentation'], 'ml_libs': ['pytorch'], 'ml_tags': ['U-Net', 'ReLu', 'NN', 'Recommender', 'CNN', 'Image Segmentation', 'AI'], 'description': "Please check out Guido's excellent kernel here. In this kernel i show how to perform inference on test set using the trained model. I just added RaDAM optimzer and got some better score. You can find the BEV of the test set here. Updates: Corrected yaw calculation Used category height information"}


In [ ]:
# iterate all folders and compose results into meta.json

url = 'https://www.kaggle.com/'
#folder_base = '../data/repositories/kaggle/competitions/'
folder_base = '../data/repositories/kaggle/datasets/'
file_notebook = 'notebook_02.html'
file_kernel = 'kernel.html'
file_out = 'meta.json'

skip = True # if true skip meta collection / set to false to recreate {file_out} from scratch
quit = 100000 # quit after n files processed
breakOnError = False

folders = os.listdir(folder_base)
print('folder:', len(folders))
i = 0
j = 0

for folder in folders:
    subfolders = os.listdir(os.path.join(folder_base,folder))
    print('subfolder:', len(folders))
    for subfolder in subfolders:
        print('###', i, os.path.join(folder_base,folder,subfolder))
        path = os.path.join(folder_base,folder,subfolder, 'notebooks/')
        i += 1
        
        if os.path.exists(path):
            projects = os.listdir(path)

            for author in projects:
                #print('author:', author)
                items = os.listdir(os.path.join(folder_base,folder,subfolder, 'notebooks/', author))

                for notebook in items:
                    j+=1
                    print(' - ', j, 'author:', author, '/ notebook:', notebook)
                    path_notebook = os.path.join(folder_base,folder,subfolder, 'notebooks/', author, notebook, file_notebook)
                    path_kernel = os.path.join(folder_base,folder,subfolder, 'notebooks/', author, notebook, file_kernel)
                    path_out = os.path.join(folder_base,folder,subfolder, 'notebooks/', author, notebook, file_out)
                    
                    meta = {}
                    
                    if skip and os.path.isfile(path_out):
                        print('skipped')
                            
                    else:
                        #try:
                        if True:
                            # get meta-data
                            if os.path.isfile(path_notebook):
                                #print('notebook found')
                                meta['scraped_at'] = creation_date(path_notebook)
                                meta['link'] = f'{url}{author}/{notebook}'
                                meta['reference'] = f'{url}{folder}/{subfolder}'
                                html = load_data(path_notebook, fromJson=False)
                                if 'file not found' in html:
                                    print('notebook not found')
                                    break
                                meta.update(scrape_notebook_content(html))

                            # get meta-data
                            if os.path.isfile(path_kernel):
                                #print('kernel found')
                                html = load_data(path_kernel, fromJson=False)
                                if 'file not found' in html:
                                    print('kernel not found')
                                    break
                                meta.update(scrape_kernel_content(html, needles))
                            else:
                                #print('kernel not found')
                                html = load_data(path_notebook, fromJson=False)
                                if 'file not found' in html:
                                    print('notebook not found')
                                    break
                                meta.update(scrape_kernel_content(html, needles))

                            #print(meta)
                            store_data(meta, path_out, toJson=True)
                            
                        #except Exception as e:
                        #    print("Oops!", e.__class__, "occurred.")
                        #    print(e)
                        #    if breakOnError:
                        #        sys.exit()
                
                if j>quit:
                    break
            if j>quit:
                break
    if j>quit:
        break

In [46]:
# scoring function to get a score between 0...1 for integer-values, 0.5 should be at ~100
def score(n, precision=3):
    if isinstance(n, int) or isinstance(i, float):
        return round(1-1/math.pow(1+n, 0.15), precision)
    else:
        try:
            n = int(n)
        except:
            return 0
        
    return round(1-1/math.pow(1+n, 0.15), precision)

for n in [0,1,10,25,50,100,1000,10000]:
    print(score(n))
    
print(score('3'))
print(score('a'))

0.0
0.099
0.302
0.387
0.446
0.5
0.645
0.749
0.188
0


In [ ]:
# throw all parsed meta-data together in a single csv
# select only true ML cases

folder_base = '../data/repositories/kaggle/competitions/'
file_json = 'meta.json'
fp_csv = '../data/database/kaggle_competitions_01_original.csv'
fp_research = '../data/database/kaggle_competitions_02_research.csv'

#datasets
folder_base = '../data/repositories/kaggle/datasets/'
fp_csv = '../data/database/kaggle_datasets_01_original.csv'
fp_research = '../data/database/kaggle_datasets_02_research.csv'

quit = 0 # quit after n files processed / 0 ... no limit

folders = os.listdir(folder_base)
print('folder:', len(folders))
i = 0
j = 0

runtime_start = time.time()
df = pd.DataFrame()
df2 = pd.DataFrame()

for folder in folders:
    subfolders = os.listdir(os.path.join(folder_base,folder))
    print('subfolder:', len(folders))
    for subfolder in subfolders:
        #print('###', i, os.path.join(folder_base,folder,subfolder))
        path = os.path.join(folder_base,folder,subfolder, 'notebooks/')
        i += 1
        
        if os.path.exists(path):
            projects = os.listdir(path)

            for author in projects:
                #print('author:', author)
                items = os.listdir(os.path.join(folder_base,folder,subfolder, 'notebooks/', author))

                for notebook in items:
                    j+=1
                    #print(' - ', j, 'author:', author, '/ notebook:', notebook)
                    fp_json = os.path.join(folder_base,folder,subfolder, 'notebooks/', author, notebook, file_json)
                    
                    if os.path.isfile(fp_json):
                        data = load_data(fp_json, fromJson=True)
                        #print(data)
                        if 'score_votes' in data:
                            data['score_votes'] = score(data['votes'])
                        if 'score_views' in data:
                            data['score_views'] = score(data['views'])
                            
                        if not 'ml_slugs' in data:
                            data['ml_slugs'] = ''
                        if not 'ml_terms' in data:
                            data['ml_terms'] = ''
                        if not 'ml_libs' in data:
                            data['ml_libs'] = ''
                        
                        ml_score = 0
                        if len(data['ml_slugs']) > 0 or len(data['ml_terms']) > 0:
                            ml_score += 0.2
                        if 'ml_tags' in data and len(data['ml_tags']) > 0:
                            ml_score += 0.3
                        if len(data['ml_libs']) > 0:
                            ml_score += 0.5
                        data['ml_detected'] = ml_score
                        
                        if 'description' in data:
                            data['description'] = data['description'].replace('\n', ' ').replace('\r', '').replace('¶', '').strip()
                        else:
                            data['description'] = ''
                        
                        if 'date' in data:
                            # date (ignoring GMT+x)
                            # Wed Dec 19 2018 14:42:40 GMT+0100 (Mitteleuropäische Normalzeit)
                            # https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
                            date_time_str = data['date'].split('GMT')
                            date_time_str = date_time_str[0].strip()
                            date_time_obj = datetime.datetime.strptime(date_time_str, '%a %b %d %Y %H:%M:%S')

                            #print('Date:', date_time_obj.date())
                            #print('Time:', date_time_obj.time())
                            #print('Date-time:', date_time_obj)
                            data['date'] = date_time_obj
                        
                        # store only items with:
                        # - ml_score >= 0.5
                        # - are indicated as "best submission" # and 'Best Submission' in data['submission'] 
                        # - description > 1 word
                        words = data['description'].split(' ')
                        if ml_score >= 0.5 and len(words) > 5:
                            df = df.append(data, ignore_index=True)
                            print(data['link'], 'is ML use case')
                        
                        else:
                            #if 'R' in data['type']:
                            df2 = df2.append(data, ignore_index=True)
                            print(data['link'], 'is not ML use case')
                            
                    #if j % 100 == 0:
                    #    print('folder', i, '/ notebook', j)
                        
                if quit!=0 and j>quit:
                    break
            if quit!=0 and j>quit:
                break
    if quit!=0 and j>quit:
        break
        
# drop duplicates
df = df.drop_duplicates(['link'])

runtime_end = time.time()
print('runtime:', round(runtime_end - runtime_start, 3), 'seconds for', j, 'items')
print(df.shape)
print(df.head())
        
# drop columns
df.drop(columns=['author', 'submission'], inplace=True)
df2.drop(columns=['author', 'submission'], inplace=True)

df.to_csv(fp_csv, sep=';', index=False)
df2.to_csv(fp_research, sep=';', index=False)

In [51]:
# print size of gathered csv
print(df.shape)
print(df2.shape)

(16730, 19)
(26744, 20)


In [52]:
# string to list

def str_to_list(s):
    s = s.replace("'", "").replace(' ,', ',').replace(
        '[', '').replace(']', '').split(',')
    s = [i for i in s if i]
    return s

In [58]:
# correlate competitions/datasets with attached notebooks

folder_base = '../data/repositories/kaggle/competitions/'
file_json = 'meta.json'
fp_csv_notebooks = '../data/database/kaggle_competitions_01_original.csv'
fp_csv_out = '../data/database/kaggle_competitions_correlated_01.csv'

# datasets
folder_base = '../data/repositories/kaggle/datasets/'
fp_csv_notebooks = '../data/database/kaggle_datasets_01_original.csv'
fp_csv_out = '../data/database/kaggle_datasets_correlated_01.csv'

quit = 0 # quit after n files processed / 0 ... no limit

folders = os.listdir(folder_base)
print('folder:', len(folders))
i = 0

runtime_start = time.time()
df = pd.DataFrame()
df_n = pd.read_csv(fp_csv_notebooks, sep=';')
#print(df_n.head)

print('folder:', len(folders))
for folder in folders:
    subfolders = os.listdir(os.path.join(folder_base,folder))
    for subfolder in subfolders:
        i += 1
        #print('###', i, os.path.join(folder_base,folder,subfolder))
        
        # read in dataset
        data = load_data(os.path.join(folder_base,folder,subfolder,file_json), fromJson=True)
        if 'title' in data:
            title = data['title'].strip()
            #print(dataset)
        else:
            data['title'] = ''
        print(i, title)
        
        #rows = []
    
        #df_n.set_index('sources', inplace = True)
        #rows = df_n.loc[title]
        #rows = df[df_n['sources'] == title]
        rows = df_n.loc[df_n['sources'] == title]
            
        #print(rows.shape[0])
        
        # if one notebook with ml is found the dataset can be considered as ml-dataset
        if rows.shape[0] > 0:
            #print('attached', data)
            
            # parse date
            if 'date_closed' in data:
                # Thu Jan 17 2019 00:59:00 GMT+0100
                date_time_str = data['date_closed'].split('GMT')
                date_time_str = date_time_str[0].strip()
                date_time_obj = datetime.datetime.strptime(date_time_str, '%a %b %d %Y %H:%M:%S')
                data['date_closed'] = date_time_obj
            
            # parse score
            
            if 'teams' in data:
                teams = data.get('teams')
                teams = int(teams.replace(',',''))
                data['teams'] = teams
                data['teams_score'] = score(data['teams'])
            else:
                data['teams_score'] = 0
            
            # parse tags
            tags = []
            for tag in rows['tags'].tolist():
                #print(tag)
                tag = str_to_list(tag)
                tags.extend(tag)
                
            tags = [x.replace('+1 more','') for x in tags]
            tags = [x.replace('+2 more','') for x in tags]
            tags = [x.replace('+3 more','') for x in tags]
            tags = [x.replace('+4 more','') for x in tags]
            tags = list(map(str.strip, tags))
            
            tags = list(set(tags))
            data['tags'] = tags
            
            #print(list(set(tags)))
            df = df.append(data, ignore_index=True)
        
        if quit!=0 and i>quit:
            break
             
    if quit!=0 and i>quit:
        break
        
# drop duplicates
#df = df.drop_duplicates(['link'])

runtime_end = time.time()
print('runtime:', round(runtime_end - runtime_start, 3), 'seconds for', j, 'items')
print(df.shape)
print(df.head())
        
# drop columns
#df.drop(columns=['description'], inplace=True)

df.to_csv(fp_csv_out, sep=';', index=False)

folder: 8467
folder: 8467
1 Computational Imaging
2 Eye Gaze
3 Finding Bubbles in Foam
4 Segmenting Soft Tissue Sarcomas
5 catdoginfo
6 Basic Arabic Vocal Emotions Dataset
7 roberta_an_an_an
8 Military Aircraft Detection Dataset
9 Bert0.01
10 bert0.2
11 bert0.3
12 Calories Burned During Exercise and Activities
13 Captcha Images
14 PUBG Weapon Stats
15 Valorant Weapon Stats
16 IPL 2008 - 2017 Predictor
17 FIA F1 (Formula 1) 1950-2020 data
18 YouTube VP and Presidential Debate Comments
19 Tarantino Scripts
20 Divorce/marriage dataset with birth dates
21 Top crypto exchanges BTC cold wallets transactions
22 Simple Source
23 Devpost Project Data
24 Jester Collaborative Filtering Dataset
25 BIG MART SALES DATASET
26 Churn_Modelling.csv
27 Saudi Hotels in Booking.com
28 AAU RainSnow Traffic Surveillance Dataset
29 AAU Zebrafish Re-Identification Dataset
30 The Brackish Dataset
31 Thermal soccer dataset
32 AAU VAP Trimodal People Segmentation Dataset
33 Demand Dataset
34 Hypertension Data
35 

282 Albert-Large-v2
283 Face Mask Dataset (YOLO  Format)
284 Attend2You - InstaPic Dataset
285 Premier League Detailed Team Data
286 Retail Analysis with Walmart Data
287 Face Recognition
288 Factors Affecting Early Adult Lung Function 2
289 Factors Affecting Early Adult Lung Function 2
290 All India Mobile Data Speed for July 2020 dataset
291 NCAA BASKETBALL MEN percentage win
292 Bike Crash Data 2007-2018
293 European Football Database 2019/2020
294 100,000 UK Used Car Data set
295 AdmissionPrediction.csv
296 Amsterdam - AirBnb
297 Latest IMDB
298 Flick 30k Dataset
299 Flickr 8k Dataset
300 Allergen Data Based on Geographic Location
301 Jojo's Bizarre Adventure
302 S&P index historical Data
303 IPL Player Stats (2008 to 2020)
304 COVID-19 Dataset for California Counties
305 COVID-19 Dataset for Michigan Counties
306 StatsBomb data
307 Births in U.S 1994 to 2003
308 IMDB movies metadata
309 Sample Stock Test
310 Koeln Strafzettel 2015
311 extradata
312 RePaUfra_v1
313 Obesity Stats
31

604 The Price and Sales of Avocado
605 ticks: bitcoin, ethereum,litecoin, ripple
606 Game of Thrones Script All Seasons
607 Laminar Coflow Flame (CRECK-C1C3HT) CH4/N2 65/35
608 Laminar Coflow Flame (DRM19) CH4/N2 65/35
609 bert-v2
610 word2vec_v2
611 word2vec-results
612 word2vec-total-results
613 AZLyrics song lyrics
614 Australian Weather Observation Data
615 google quest lstm model trained in keras
616 Data_files
617 Model Zoo utility files for object detection task
618 Philippine Voters Profile
619 Saudi Daily Stocks History (Tadawul)
620 Facebook Anti-Vaccine Post Data (Scaled Features)
621 Panda_Dataset_final_36_256_256
622 La Liga Santander 2019-2020
623 User's Age by Expenses
624 Rent Prices - Spain
625 Industrial Production Index in USA
626 Leading Indicators OECD
627 Trade of Goods Indexes by Country
628 World Bank's Global Economic Prospects Data
629 World Economic Indiactors by OECD
630 World National and Real GDP (Annualy/Quaterly)
631 World Unemployment Rate from OECD
632

890 12 Months Financial Data
891 Análisis Mercado Libre - Categoría Refrigeradores
892 Fatal US Police Shootings Data
893 Youtube Video Statistics
894 Solar and Meter Readings 15mins
895 Titanic
896 history_weather_munich
897 Linear Regression
898 2020 Cost of Living
899 Bing, NRC, Afinn Lexicons
900 Birdcall Recognition Data
901 Country Mapping - ISO, Continent, Region
902 GTZAN Dataset - Music Genre Classification
903 ISO Country Codes - Global
904 Pulmonary Fibrosis Prep Data
905 Rick&Morty Scripts
906 Riiid! Answer Correctness Prediction - rapids
907 Estrutura Educacional X Desempenho ENEM
908 Twitch Social Networks
909 Spanish Poetry Dataset
910 Swedish NER corpus
911 International airline passengers
912 Acres burned in forest fires in Canada, 1918-1988
913 breast_cancer
914 Duke Breast Cancer Dataset
915 Sales Of Shampoo
916 Species Classification
917 Campeonato Brasileiro 2009-2018
918 resnest269
919 Data Jobs Listings - Glassdoor
920 COVID-19 Mexico Clean & Order by States
921 

1140 Milan Airbnb Open Data (only entire apartments)
1141 Words in Portuguese (BR) and Emotions
1142 COVID-19 Community Data Collection
1143 Malicious_n_Non-Malicious URL
1144 Hotel review
1145 Spooky Author
1146 Indian Economical Data  1990 to 2019
1147 Car damage detection
1148 Uber Request Data.csv
1149 emotion_analysis
1150 Liver_patient
1151 Million Song Data Set Subset
1152 Mobile Data Speeds of All India during March, 2018
1153 Alphabet Stocks
1154 Restaurant_Reviews
1155 newdata
1156 DeepSlice & Secure5G - 5G & LTE Wireless Dataset
1157 Data Analytics to study music streaming patterns
1158 cells_recog
1159 NETWORK ANAMOLY DETECTION
1160 california housing value
1161 Recruit Restaurant Visitor Forecasting Data
1162 ocr_test
1163 ensemble3
1164 American Sexual Health Association
1165 Cotas Atividade Parlamentar dos Senadores (CEAPS)
1166 nepal json file
1167 NLL Statistics
1168 San Francisco Building Permits
1169 The Paleobiology Database
1170 Mental Health Dataset
1171 Profile o

...

15387 Flights and Airports Data
15388 Melbourne Airbnb Open Data
15389 Bike Ads (images, prices, specifications)
15390 Ice Cream Dataset
15391 xlnetmodel
15392 countries.geo.json
15393 opsenx
15394 steel-fpn-b4
15395 Amazon sales rank data for print and kindle books
15396 Dice: d4, d6, d8, d10, d12, d20 Images
15397 Anna University - Results Dataset
15398 Adult Census Income
15399 Air pressure system failures in Scania trucks
15400 Auto-mpg dataset
15401 Bioassay Datasets
15402 Biomechanical features of orthopedic patients
15403 Breast Cancer Wisconsin (Diagnostic) Data Set
15404 Caravan Insurance Challenge
15405 CAT Scan Localization
15406 Default of Credit Card Clients Dataset
15407 El Nino Dataset
15408 Household Electric Power Consumption
15409 Faulty Steel Plates
15410 Forest Cover Type Dataset
15411 German Credit Risk
15412 Glass Classification
15413 Horse Colic Dataset
15414 Human Activity Recognition with Smartphones
15415 Identifying Interesting Web Pages
15416 Indian Liver Pa

15662 Coronavirus Cases Karnataka
15663 coronavirus-cases-in-india
15664 DSL Corpus Collection (DSLCC)
15665 IEEE COVID-19 Tweets Dataset
15666 Transcripts Press Conferences NL on COVID-19
15667 H1B 2016
15668 Month-wise COVID-19 related tweets
15669 BERTModel
15670 BIOBERTCODE
15671 biobertconfig
15672 biobertmodel2
15673 Code repo for all code
15674 ZippedFolder
15675 imdb Dataset
15676 Employee Salary Dataset
15677 email msg spam detection
15678 Starwars Survey in USA
15679 SpringOpen 406 Books Details
15680 WikiHow Summarization
15681 Pisa 2018 Worldwide Ranking
15682 Sweden COVID-19 Data
15683 UK COVID-19 Data
15684 quora question pair
15685 DAX-index
15686 dax-index2
15687 FTSE 100 index
15688 FTSE 100 index2
15689 ftse100
15690 ftse1-predictions
15691 gold_new
15692 gold_new3
15693 gold_new4
15694 GOLD-index
15695 index_sp
15696 new_data1
15697 sales_train
15698 S&P_500
15699 S&P500
15700 sp500_index
15701 Skylines 12
15702 Predicting Coupon Redemption
15703 Face Recognition Dat

15943 Run or Walk (reduced)
15944 go-nuts archive
15945 California Study
15946 LC50 Data
15947 Okun's Law within the United States
15948 Boba Shops in the Bay Area
15949 wheat-spike-dataset
15950 wheat-dataset-original
15951 lossData
15952 bert_en_cased_L-12_H-768_A-12_1
15953 Bank marketing campaigns dataset | Opening Deposit
15954 Heart_Disease
15955 Coffee Quality database from CQI
15956 Temperature Time-Series for some Brazilian cities
15957 Bolsas da Capes
15958 Weather Istanbul Data 2009-2019
15959 Congressional Voting Records
15960 Nails segmentation
15961 kaggle-porto-seguro-submissions
15962 Annotated NER for Indian language
15963 Janatahack: Customer Segmentation
15964 Datasets used in my study of target encodings
15965 createdbyvrptester
15966 CVE (Common Vulnerabilities and Exposures)
15967 Classified Ads for Cars - unique maker/model/year
15968 Ukraine Register of Individual Entrepreneurs
15969 FastText
15970 Sports Car Choice data
15971 Disaster Tweets
15972 UARU_Str_DB
1

16230 Michael Jordan, Kobe Bryant and Lebron James stats
16231 Star Wars Movie Scripts
16232 The Cure discography
16233 Tweets during Cavaliers vs Warriors
16234 Tweets during Nintendo E3 2018 Conference
16235 Tweets during Real Madrid vs Liverpool
16236 Dataset malware/beningn permissions Android
16237 Malicious and Benign Websites
16238 Network Traffic Android Malware
16239 Cell_segmentation
16240 Steam Sale
16241 Australia Saturday Xlotto Jackpot Numbers
16242 Turkey COVID 19 Complete Dataset
16243 FIFA 21 messy, raw dataset for cleaning/ exploring
16244 Bangladesh Weather Dataset
16245 Most Popular Soccer Leagues
16246 Spotify Dataset 1921-2020, 160k+ Tracks
16247 World Happiness Report (Preprocessed)
16248 RIIID Questions, tags, lectures expanded metadata
16249 Exchange Rates
16250 Top8USACities_PropertySales
16251 NYC Uber Pickups with Weather and Holidays
16252 modelchexnet
16253 bart_code
16254 bart_large
16255 bert_base_pretrained
16256 fairseq_hacked
16257 gpt2bpe
16258 quest

16509 Python package Datatable
16510 A/B testing
16511 Final_Capsule
16512 Final_EfficientNetB5
16513 TitanicDatasets
16514 Defects location for metal surface
16515 driving data
16516 Heart.csv
16517 Dhaka Stock Exchange Broad Historical Data
16518 Global Aquaculture Imports and Exports
16519 wine-price-rating
16520 toxic_comments_fastText
16521 Loan  Data
16522 Technical Indicator Backtest
16523 bert_pretrained_model
16524 bert_pretrained_model_uncased
16525 tensorflow_hub_bert
16526 omegaconf
16527 ernie_model
16528 model_src
16529 rbt3_chinese_bert
16530 roberta-large-wwm-chinese
16531 bank_data_loan_default
16532 lajitraindata
16533 The World Factbook by CIA
16534 United Nations Crime-Data
16535 Divorce dataset
16536 Median Listing Price (1 Bedroom)
16537 Zillow Rent Index, 2010-Present
16538 Zillow Economics Data
16539 Gold Prices London Market 1950 - Present
16540 IATA Airport Codes
16541 Major World Cities in the World
16542 S&P 500 Companies with Financial Information
16543 COV